<a href="https://colab.research.google.com/github/abyaadrafid/APTOS-Blindness-Detection/blob/master/autoencoder_self_supervision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install geopandas

In [262]:
import torch
import geopandas as gpd
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageDraw
from osgeo import gdal
import numpy as np
import math
from tqdm import notebook
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision import transforms, utils
import torchvision.models as models
import torch.nn as nn
from torch.optim import Adam
from google.colab.patches import cv2_imshow
from itertools import product

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import os
rootdir = "/content/drive/My Drive/tiles_updated/tiles/"

In [17]:
samples = os.listdir(rootdir)

In [18]:
def cooord2pixel(xp, yp, geo_transform):
  xoff, a, b, yoff, d, e  = geo_transform 
  y = (a*yp - d*xp + d*xoff - a*yoff) / (a*e - d*b)
  x = (xp - xoff - b * y)/a
  return x, y

In [19]:
def get_locations(sample):

  image_location = rootdir+sample+'/'+sample+'_RGBI.tif'
  annotation_location = rootdir+sample+'/'+ 'results' + sample[12:]+'/tree_crown_poly_smooth.shp'

  return image_location, annotation_location

In [57]:
def preprocess_dataset(samples) :
  
  data_points = []

  for i,_ in notebook.tqdm(enumerate(samples)) :
    
    image_location, annotation_location = get_locations(samples[i])
    gdf = gpd.GeoDataFrame.from_file(annotation_location)
    ds = gdal.Open(image_location).GetGeoTransform()

    for item in range(len(gdf)):

      x_coords, y_coords = gdf.geometry[item].exterior.xy
      x_coords = np.array(x_coords)
      y_coords = np.array(y_coords)

      x_points, y_points = cooord2pixel(x_coords, y_coords, ds)
      data_points.append((x_points,y_points,image_location))

  return data_points

In [105]:
def crop_image(img):
  mask = img!=0
  mask = mask.any(2)
  mask0,mask1 = mask.any(0),mask.any(1)
  return img[np.ix_(mask1,mask0)]

In [106]:
data_list = preprocess_dataset(samples)

0it [00:00, ?it/s]

In [267]:
def get_polygon(params):

  x_points, y_points, image_location = params
  img = Image.open(image_location)
  image = np.array(img)

  polygon = np.concatenate((x_points.reshape(-1,1),y_points.reshape(-1,1)),axis=1)
  polygon = polygon.astype(int)
  mask = np.zeros((image.shape[0], image.shape[1]))
  cv2.fillConvexPoly(mask, polygon, 1)
  mask = mask.astype(np.bool)
  out = np.zeros_like(image)
  out[mask] = image[mask]

  out = crop_image(out)

  if out.size!=0 :
   out = cv2.resize(out,(96, 96))
  
  return out

In [268]:
class PinaDataset(Dataset):

  def __init__(self, data_list):
    self.data_list = data_list
  
  def __len__(self):
    return len(self.data_list)

  def __getitem__(self, idx):
    polygon = get_polygon(data_list[idx])
    return torchvision.transforms.functional.to_tensor(polygon)

In [269]:
pina_ds = PinaDataset(data_list)

In [270]:
pina_ds[0].shape

torch.Size([4, 96, 96])

In [271]:
train_ds, valid_ds = random_split(pina_ds,[50895, 12723])

In [272]:
train_dl = DataLoader(train_ds, batch_size=2, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=2, shuffle=True)

In [273]:
model = nn.Sequential(
    nn.Conv2d(4, 16, kernel_size=(3,3))
)

In [274]:
for count, i in enumerate(train_dl) :
  print(model(i).shape)
  if count >= 3:
    break

torch.Size([2, 16, 94, 94])
torch.Size([2, 16, 94, 94])
torch.Size([2, 16, 94, 94])
torch.Size([2, 16, 94, 94])
